https://github.com/Noah0x0/lambda-waterLevelAPI/blob/master/lambda_function.py

In [43]:
import boto3
import json
import pandas as pd

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [5]:
client = boto3.client('s3')
bucket = 'test-uodu-s3'
#target = 'waterLevel/japan/ishikawa/asano/2017'
target = 'waterLevel/japan/ishikawa/asano/2017/09/'

In [6]:
response = client.list_objects(
    Bucket=bucket,
    Prefix=target
)
if 'Contents' in response:  # 該当する key がないと response に 'Contents' が含まれない
    keys = [content['Key'] for content in response['Contents']]
len(keys)

1000

In [7]:
def get_all_keys(bucket: str='', prefix: str='', keys: []=[], marker: str='') -> [str]:
    """
    指定した prefix のすべての key の配列を返す
    """
    response = client.list_objects(Bucket=bucket, Prefix=prefix, Marker=marker)
    if 'Contents' in response:  # 該当する key がないと response に 'Contents' が含まれない
        keys.extend([content['Key'] for content in response['Contents']])
        if 'IsTruncated' in response:
            return get_all_keys(bucket=bucket, prefix=prefix, keys=keys, marker=keys[-1])
    return keys

keyta = get_all_keys(bucket, target)
#keyta

In [8]:
keyta[2:3]

['waterLevel/japan/ishikawa/asano/2017/09/01/00:20:00.json']

In [9]:
len(keyta)

3876

In [15]:
response = client.get_object(
    Bucket=bucket,
    Key="waterLevel/japan/ishikawa/asano/2017/09/01/00:20:00.json"
)
body = response['Body'].read().decode('utf-8')
body

'{"riverName":"浅野川","height":"5.50","timestamp":"2017-09-01T00:20:00Z","waterLevel":"0.26","dataTrend":"→","dataLevel":0,"observatory":"金沢市沖橋(諸江)"}'

In [16]:
jsondic = json.loads(body)
jsondic

{'dataLevel': 0,
 'dataTrend': '→',
 'height': '5.50',
 'observatory': '金沢市沖橋(諸江)',
 'riverName': '浅野川',
 'timestamp': '2017-09-01T00:20:00Z',
 'waterLevel': '0.26'}

In [17]:
response2 = client.get_object(
    Bucket=bucket,
    Key="waterLevel/japan/ishikawa/asano/2017/09/01/00:30:00.json")
body2 = response2['Body'].read().decode('utf-8')
jsondic2 = json.loads(body2)
jsondic2

{'dataLevel': 0,
 'dataTrend': '↑',
 'height': '5.50',
 'observatory': '金沢市沖橋(諸江)',
 'riverName': '浅野川',
 'timestamp': '2017-09-01T00:30:00Z',
 'waterLevel': '0.27'}

In [18]:
jsons = []
jsons.append(jsondic)
jsons.append(jsondic2)
jsons

[{'dataLevel': 0,
  'dataTrend': '→',
  'height': '5.50',
  'observatory': '金沢市沖橋(諸江)',
  'riverName': '浅野川',
  'timestamp': '2017-09-01T00:20:00Z',
  'waterLevel': '0.26'},
 {'dataLevel': 0,
  'dataTrend': '↑',
  'height': '5.50',
  'observatory': '金沢市沖橋(諸江)',
  'riverName': '浅野川',
  'timestamp': '2017-09-01T00:30:00Z',
  'waterLevel': '0.27'}]

In [19]:
df_level = pd.DataFrame(jsons)
df_level.head()

,dataLevel,dataTrend,height,observatory,riverName,timestamp,waterLevel
0,0,→,5.50,金沢市沖橋(諸江),浅野川,2017-09-01T00:20:00Z,0.26
1,0,↑,5.50,金沢市沖橋(諸江),浅野川,2017-09-01T00:30:00Z,0.27


In [21]:
jsons = []
for key in keyta:
    response = client.get_object(Bucket=bucket, Key=key)
    body = response['Body'].read().decode('utf-8')
    jsondict = json.loads(body)
    del jsondict["dataLevel"]
    del jsondict["dataTrend"]
    del jsondict["height"]
    del jsondict["observatory"]
    del jsondict["riverName"]
    jsons.append(jsondict)
len(jsons)

3876

In [25]:
df_level = pd.DataFrame(jsons)
df_level.columns = ["datetime","level"]
df_level.tail()

,datetime,level
3871,2017-09-30T23:10:00Z,0.24
3872,2017-09-30T23:20:00Z,0.24
3873,2017-09-30T23:30:00Z,0.24
3874,2017-09-30T23:40:00Z,0.24
3875,2017-09-30T23:50:00Z,0.23


In [28]:
#f = open("kWaterLevel.json", "w")
#json.dump(jsons, f, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))

In [41]:
sep_json = df_level.to_json(orient='records')
#sep_json

In [47]:
json_dict = json.loads(sep_json)

In [57]:
f = open("waterLevelSeptember.json", "w")
json.dump(json_dict, f, indent=4)
#json.dump(json_dict, f, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))